## Imports and Initialisations

In [16]:
import nltk
nltk.download('stopwords')

from nltk.corpus import brown, stopwords
import xml.etree.ElementTree as ET

sents = brown.sents()
stopwords = set(stopwords.words('english'))
tree = ET.parse('../data/test/subtask1-homographic-test.xml')
puncts = []
test_sent = []

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arghyabhattacharya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
F = open("../data/test/subtask1-homographic-test.gold")
for item in F.readlines():
    ans = item.split()
    

['hom_1', '1']
['hom_2', '1']
['hom_3', '1']
['hom_4', '1']
['hom_5', '1']
['hom_6', '0']
['hom_7', '1']
['hom_8', '1']
['hom_9', '1']
['hom_10', '1']
['hom_11', '1']
['hom_12', '0']
['hom_13', '0']
['hom_14', '1']
['hom_15', '1']
['hom_16', '1']
['hom_17', '0']
['hom_18', '1']
['hom_19', '1']
['hom_20', '1']
['hom_21', '1']
['hom_22', '1']
['hom_23', '1']
['hom_24', '1']
['hom_25', '1']
['hom_26', '1']
['hom_27', '1']
['hom_28', '1']
['hom_29', '1']
['hom_30', '1']
['hom_31', '1']
['hom_32', '0']
['hom_33', '1']
['hom_34', '1']
['hom_35', '0']
['hom_36', '0']
['hom_37', '0']
['hom_38', '1']
['hom_39', '1']
['hom_40', '0']
['hom_41', '1']
['hom_42', '0']
['hom_43', '1']
['hom_44', '1']
['hom_45', '1']
['hom_46', '1']
['hom_47', '1']
['hom_48', '1']
['hom_49', '0']
['hom_50', '1']
['hom_51', '1']
['hom_52', '1']
['hom_53', '1']
['hom_54', '0']
['hom_55', '1']
['hom_56', '0']
['hom_57', '0']
['hom_58', '1']
['hom_59', '0']
['hom_60', '1']
['hom_61', '1']
['hom_62', '1']
['hom_63', '1']
[

['hom_1780', '1']
['hom_1781', '1']
['hom_1782', '1']
['hom_1783', '1']
['hom_1784', '0']
['hom_1785', '1']
['hom_1786', '1']
['hom_1787', '1']
['hom_1788', '0']
['hom_1789', '0']
['hom_1790', '1']
['hom_1791', '1']
['hom_1792', '1']
['hom_1793', '0']
['hom_1794', '1']
['hom_1795', '1']
['hom_1796', '1']
['hom_1797', '1']
['hom_1798', '1']
['hom_1799', '1']
['hom_1800', '0']
['hom_1801', '1']
['hom_1802', '0']
['hom_1803', '1']
['hom_1804', '1']
['hom_1805', '1']
['hom_1806', '1']
['hom_1807', '1']
['hom_1808', '1']
['hom_1809', '1']
['hom_1810', '0']
['hom_1811', '0']
['hom_1812', '1']
['hom_1813', '1']
['hom_1814', '1']
['hom_1815', '1']
['hom_1816', '0']
['hom_1817', '0']
['hom_1818', '0']
['hom_1819', '1']
['hom_1820', '1']
['hom_1821', '1']
['hom_1822', '1']
['hom_1823', '1']
['hom_1824', '1']
['hom_1825', '1']
['hom_1826', '1']
['hom_1827', '0']
['hom_1828', '1']
['hom_1829', '0']
['hom_1830', '1']
['hom_1831', '1']
['hom_1832', '1']
['hom_1833', '1']
['hom_1834', '1']
['hom_1835

In [ ]:
root = tree.getroot()
for text in root.getchildren():
    sent = list()
    for word in text.getchildren():
        sent.append(word.text)
    sent = str(sent)
    test_sent.append(sent)
    

In [ ]:
def make_ordered_pairs(sentence):
    ans = list()
    for index, word in enumerate(sentence):
        temp = index
        while(temp != len(sentence) - 1):
            ans.append((sentence[index], sentence[temp]))
            temp += 1
    return ans

In [ ]:
for sent in sents[:20]:
    filtered_sent = [w for w in sent if not w.lower() in stopwords]
    pairs = make_ordered_pairs(filtered_sent)
    print(pairs)